This code section aimed to use to create the database GDSC by combining the selfies and smiles code to the drug, and also connect the cell line name to depmap ID

In [2]:
#read the csv LUADandLUSC_IC_DrugCCL_IC50.csv
import pandas as pd 
import numpy as np 
import selfies as sf 
import pubchempy as pcp 

In [3]:
CCLandDrugIC50_path = 'dataset/LUADandLUSC_IC_DrugCCL_IC50.csv'
CCLandDrugIC50 = pd.read_csv(CCLandDrugIC50_path)
#print CCLandDrugIC50 top 5 rows
CCLandDrugIC50.head()


,Drug Name,Drug ID,Cell Line Name,Cosmic ID,TCGA Classification,Tissue,Tissue Sub-type,IC50,AUC,Max Conc,RMSE,Z score,Dataset Version
0,Camptothecin,1003,Calu-3,687777,LUAD,lung,lung_NSCLC_adenocarcinoma,-1.629546,0.863243,0.1,0.096562,0.342928,GDSC2
1,Camptothecin,1003,NCI-H1437,687794,LUAD,lung,lung_NSCLC_adenocarcinoma,-1.839524,0.900130,0.1,0.095318,0.228601,GDSC2
2,Camptothecin,1003,NCI-H1623,687798,LUAD,lung,lung_NSCLC_adenocarcinoma,-4.135944,0.687601,0.1,0.152799,-1.021727,GDSC2
3,Camptothecin,1003,NCI-H1648,687799,LUAD,lung,lung_NSCLC_adenocarcinoma,-1.962265,0.879493,0.1,0.108430,0.161773,GDSC2
4,Camptothecin,1003,NCI-H1650,687800,LUAD,lung,lung_NSCLC_adenocarcinoma,0.133024,0.936097,0.1,0.075301,1.302591,GDSC2


In [4]:
Drug_SMILES_SELFIES_path = 'dataset/LUAD_LUSC_drug_finalPreprocess.csv'
Drug_SMILES_SELFIES = pd.read_csv(Drug_SMILES_SELFIES_path)
#print Drug_SMILES_SELFIES top 5 rows
Drug_SMILES_SELFIES.head()

,Drug name,Feature Name,SMILES,SELFIES,Drug ID,ic50_effect_size,Tissue Type
0,Camptothecin,ARID1A_mut,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,[C][C][C][Branch2][Ring2][=Branch1][C][=C][Bra...,1003,0.177968,LUAD
1,Camptothecin,EGFR_mut,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,[C][C][C][Branch2][Ring2][=Branch1][C][=C][Bra...,1003,0.352381,LUAD
2,Camptothecin,KDR_mut,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,[C][C][C][Branch2][Ring2][=Branch1][C][=C][Bra...,1003,0.892665,LUAD
3,Camptothecin,KRAS_mut,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,[C][C][C][Branch2][Ring2][=Branch1][C][=C][Bra...,1003,0.151599,LUAD
4,Camptothecin,MGA_mut,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,[C][C][C][Branch2][Ring2][=Branch1][C][=C][Bra...,1003,0.154535,LUAD


In [5]:
#delete the duplicate rows in Drug_SMILES_SELFIES
Drug_SMILES_SELFIES = Drug_SMILES_SELFIES.drop_duplicates(subset=['Drug name'])
#reset the index of Drug_SMILES_SELFIES
Drug_SMILES_SELFIES = Drug_SMILES_SELFIES.reset_index(drop=True)
Drug_SMILES_SELFIES = Drug_SMILES_SELFIES.drop(columns=['Feature Name','Tissue Type','ic50_effect_size'])   

In [6]:
Drug_SMILES_SELFIES = Drug_SMILES_SELFIES.drop(columns=['Drug ID'])   
Drug_SMILES_SELFIES.head()

,Drug name,SMILES,SELFIES
0,Camptothecin,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,[C][C][C][Branch2][Ring2][=Branch1][C][=C][Bra...
1,Vinblastine,CCC1(CC2CC(C3=C(CCN(C2)C1)C4=CC=CC=C4N3)(C5=C(...,[C][C][C][Branch2][Branch2][O][C][C][C][C][Bra...
2,Cisplatin,N.N.Cl[Pt]Cl,[N].[N].[Cl][Pt][Cl]
3,Cytarabine,C1=CN(C(=O)N=C1N)C2C(C(C(O2)CO)O)O,[C][=C][N][Branch1][#Branch2][C][=Branch1][C][...
4,Docetaxel,CC1=C2C(C(=O)C3(C(CC4C(C3C(C(C2(C)C)(CC1OC(=O)...,[C][C][=C][C][Branch2][Branch2][N][C][=Branch1...


In [7]:
CCLandDrugIC50 = CCLandDrugIC50.rename(columns={'Drug Name':'Drug name'})


In [8]:
#merge Drug_SMILES_SELFIES.csv and CCLandDrugIC50 file on 'Drug name' to add SMILES and SELFIES columns
CCLandDrugIC50 = pd.merge(CCLandDrugIC50, Drug_SMILES_SELFIES, on='Drug name', how='left')

In [9]:
# current CCLandDrugIC50 file with SMILES and SELFIES columns (Drug) to the cell line name -> IC50 value
CCLandDrugIC50.head(2)

,Drug name,Drug ID,Cell Line Name,Cosmic ID,TCGA Classification,Tissue,Tissue Sub-type,IC50,AUC,Max Conc,RMSE,Z score,Dataset Version,SMILES,SELFIES
0,Camptothecin,1003,Calu-3,687777,LUAD,lung,lung_NSCLC_adenocarcinoma,-1.629546,0.863243,0.1,0.096562,0.342928,GDSC2,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,[C][C][C][Branch2][Ring2][=Branch1][C][=C][Bra...
1,Camptothecin,1003,NCI-H1437,687794,LUAD,lung,lung_NSCLC_adenocarcinoma,-1.839524,0.900130,0.1,0.095318,0.228601,GDSC2,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,[C][C][C][Branch2][Ring2][=Branch1][C][=C][Bra...


In [13]:
# now I will insert the 'Depmap Id' to the file CCLandDrugIC50 based on their Cell Line Name
CCL_list_path = 'dataset/cell lines in Lung.csv'
CCL_list = pd.read_csv(CCL_list_path)
#drop column Unnamed:4 and Unnamed:5
CCL_list = CCL_list.drop(columns=['Unnamed: 4','Unnamed: 5'])
CCL_list.head()


,Depmap Id,Cell Line,Primary Disease,Tumor Type
0,ACH-000729,NCIH1963,Lung Neuroendocrine Tumor,Primary
1,ACH-001549,LU135,Lung Neuroendocrine Tumor,Primary
2,ACH-000860,NCIH358,Non-Small Cell Lung Cancer,Primary
3,ACH-000553,SQ1,Non-Small Cell Lung Cancer,Primary
4,ACH-000186,NCIH2444,Non-Small Cell Lung Cancer,Primary


In [20]:
CCL_list = CCL_list.rename(columns={'Cell Line':'Cell Line Name'})

In [ ]:
# problem is that the cell line name in CCLE and GDSC label differently. NCI-H358 (GDSC) = NCIH538 (CCLE). or COR-L105 (GDSC) = COR-L105 (CCLE)
# so, we need to preprocess the '-' in the GDSC (IC50) file first 

# remove '-' in the Cell Line Name in CCLandDrugIC50 file
CCLandDrugIC50['Cell Line Name'] = CCLandDrugIC50['Cell Line Name'].str.replace('-', '')
CCLandDrugIC50.head(2)


In [22]:
#merge CCL_list and CCLandDrugIC50 on 'Cell Line Name' to add Depmap Id column
CCLandDrugIC50 = pd.merge(CCLandDrugIC50, CCL_list, on='Cell Line Name', how='left')
CCLandDrugIC50.head(5)

,Drug name,Drug ID,Cell Line Name,Cosmic ID,TCGA Classification,Tissue,Tissue Sub-type,IC50,AUC,Max Conc,RMSE,Z score,Dataset Version,SMILES,SELFIES,Depmap Id,Primary Disease,Tumor Type
0,Camptothecin,1003,Calu3,687777,LUAD,lung,lung_NSCLC_adenocarcinoma,-1.629546,0.863243,0.1,0.096562,0.342928,GDSC2,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,[C][C][C][Branch2][Ring2][=Branch1][C][=C][Bra...,NaN,NaN,NaN
1,Camptothecin,1003,NCIH1437,687794,LUAD,lung,lung_NSCLC_adenocarcinoma,-1.839524,0.900130,0.1,0.095318,0.228601,GDSC2,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,[C][C][C][Branch2][Ring2][=Branch1][C][=C][Bra...,ACH-000589,Non-Small Cell Lung Cancer,Metastatic
2,Camptothecin,1003,NCIH1623,687798,LUAD,lung,lung_NSCLC_adenocarcinoma,-4.135944,0.687601,0.1,0.152799,-1.021727,GDSC2,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,[C][C][C][Branch2][Ring2][=Branch1][C][=C][Bra...,ACH-000744,Non-Small Cell Lung Cancer,Metastatic
3,Camptothecin,1003,NCIH1648,687799,LUAD,lung,lung_NSCLC_adenocarcinoma,-1.962265,0.879493,0.1,0.108430,0.161773,GDSC2,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,[C][C][C][Branch2][Ring2][=Branch1][C][=C][Bra...,ACH-000766,Non-Small Cell Lung Cancer,Metastatic
4,Camptothecin,1003,NCIH1650,687800,LUAD,lung,lung_NSCLC_adenocarcinoma,0.133024,0.936097,0.1,0.075301,1.302591,GDSC2,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...,[C][C][C][Branch2][Ring2][=Branch1][C][=C][Bra...,ACH-000035,Non-Small Cell Lung Cancer,Metastatic


In [23]:
#save the CCLandDrugIC50 file to csv
CCLandDrugIC50.to_csv('dataset/4_LUADandLUSC_DrugCCL_withDepmapId.csv', index=False)